# Covid-19 data analysis for the Netherland
## uses covid data to learn datascience in pyhton with Jupyterlab

*Goals*
1. learning datawcience
2. learning python
3. learning Jupyterlab
4. maybe get some insight in covid

In [1]:
## install modules for Google Colab
# uncomment line below and run cell
#!pip install tabula-py cbsodata

In [1]:
#from IPython.display import Markdown as md
import os
import pandas as pd
import cbsodata as cbs
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib widget
import tabula

#import numpy as np

## Revert to charts in pandas, for simplicity. 
#To DO: add Bokeh - https://docs.bokeh.org/
#from bokeh.io import output_notebook
#from bokeh.plotting import figure, show
#from bokeh.palettes import viridis
#import pandas_bokeh
#output_notebook()

## Collecting data

### CBS
https://opendata.cbs.nl/statline/portal.html

In [7]:
print('Importing: Bevolkingsontwikkeling; maand en jaar 2002-now (Population statistics, month and year)')
cbs_data_loc = './data/cbs_population.pkl'
try:
    os.stat(cbs_data_loc)
    print('try')
except:
    pd.DataFrame(cbs.get_data('37230ned')).to_pickle(cbs_data_loc)
    print('except')
finally:
    population = pd.read_pickle(cbs_data_loc)
print('DONE')
population

Importing: Bevolkingsontwikkeling; maand en jaar 2002-now (Population statistics, month and year)
except
DONE


,ID,RegioS,Perioden,BevolkingAanHetBeginVanDePeriode_1,LevendGeborenKinderen_2,Overledenen_3,TotaleVestiging_4,VestigingVanuitEenAndereGemeente_5,Immigratie_6,TotaalVertrekInclAdmCorrecties_7,VertrekNaarAndereGemeente_8,EmigratieInclusiefAdmCorrecties_9,OverigeCorrecties_10,Bevolkingsgroei_11,BevolkingsgroeiRelatief_12,BevolkingsgroeiSinds1Januari_13,BevolkingsgroeiSinds1JanuariRela_14,BevolkingAanHetEindeVanDePeriode_15
0,0,Nederland,2002 januari,16105285.0,17019.0,13469.0,66547.0,55181.0,11366.0,62482.0,55181.0,7301.0,NaN,7615.0,0.05,7615.0,0.05,16112900.0
1,1,Nederland,2002 februari,16112900.0,15448.0,11735.0,55999.0,47049.0,8950.0,53659.0,47049.0,6610.0,NaN,6053.0,0.04,13668.0,0.08,16118953.0
2,2,Nederland,2002 maart,16118953.0,16792.0,13281.0,58798.0,49290.0,9508.0,57137.0,49290.0,7847.0,NaN,5172.0,0.03,18840.0,0.12,16124125.0
3,3,Nederland,2002 april,16124125.0,15995.0,11968.0,55730.0,46464.0,9266.0,53636.0,46464.0,7172.0,NaN,6121.0,0.04,24961.0,0.15,16130246.0
4,4,Nederland,2002 mei,16130246.0,16800.0,11623.0,58679.0,49782.0,8897.0,57295.0,49782.0,7513.0,NaN,6561.0,0.04,31522.0,0.20,16136807.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147739,147739,Zwolle,2020 mei,129038.0,139.0,59.0,487.0,463.0,24.0,453.0,408.0,45.0,NaN,114.0,0.09,312.0,0.24,129152.0
147740,147740,Zwolle,2020 juni,129152.0,97.0,72.0,457.0,428.0,29.0,492.0,488.0,4.0,NaN,-10.0,-0.01,302.0,0.23,129142.0
147741,147741,Zwolle,2020 juli,129142.0,131.0,61.0,639.0,586.0,53.0,638.0,536.0,102.0,NaN,71.0,0.05,373.0,0.29,129213.0
147742,147742,Zwolle,2020 augustus,129213.0,120.0,74.0,654.0,586.0,68.0,561.0,516.0,45.0,NaN,139.0,0.11,512.0,0.40,129352.0


In [4]:
print('Preparing Dataframe for analyses')
print('Dropping unnecessary columns')
population = population.drop(columns=['ID', 'BevolkingAanHetBeginVanDePeriode_1','TotaleVestiging_4',
       'VestigingVanuitEenAndereGemeente_5', 'Immigratie_6',
       'TotaalVertrekInclAdmCorrecties_7', 'VertrekNaarAndereGemeente_8',
       'EmigratieInclusiefAdmCorrecties_9', 'OverigeCorrecties_10',
       'Bevolkingsgroei_11', 'BevolkingsgroeiRelatief_12',
       'BevolkingsgroeiSinds1Januari_13',
       'BevolkingsgroeiSinds1JanuariRela_14'])
population = population.rename(columns={"LevendGeborenKinderen_2": "Born", "Overledenen_3": "Deceased", "BevolkingAanHetEindeVanDePeriode_15": "Population"})
print('DONE')

Preparing Dataframe for analyses
Dropping unnecessary columns
DONE


In [5]:
# defining function for calculating percentage
def percentage(df,d,n):
    df[d+'_perc'] = df[d]/df[n]*100

In [6]:
print('Setting PeriodIndex')
population['year'] = population.loc[:,'Perioden'].str.slice(stop=4)
population['month'] = population.loc[:,'Perioden'].str.slice(start=5)
population_y = population.loc[population['month'] == '',:]
population_y = population_y.drop(columns='month')
population_y = population_y.set_index(population_y['year'])
population_y.index = pd.PeriodIndex(population_y.index,freq='Y')

population = population.loc[population['month'] != '',:]
population['month'] = population.loc[:,'month'].replace({
    'januari': '01',
    'februari': '02',
    'maart': '03',
    'april': '04',
    'mei': '05',
    'juni': '06',
    'juli': '07',
    'augustus': '08',
    'september': '09',
    'oktober': '10',
    'november': '11',
    'december': '12'})

population = population.set_index(population['year'] + '-' + population['month'])
population.index = pd.PeriodIndex(population.index, freq='M')
print('DONE')

Setting PeriodIndex
DONE


In [7]:
print(population_y)

         RegioS Perioden      Born  Deceased  Population  year
year                                                          
2002  Nederland     2002  202083.0  142355.0  16192572.0  2002
2003  Nederland     2003  200297.0  141936.0  16258032.0  2003
2004  Nederland     2004  194007.0  136553.0  16305526.0  2004
2005  Nederland     2005  187910.0  136402.0  16334210.0  2005
2006  Nederland     2006  185057.0  135372.0  16357992.0  2006
...         ...      ...       ...       ...         ...   ...
2015     Zwolle     2015    1491.0     967.0    124896.0  2015
2016     Zwolle     2016    1541.0     913.0    125548.0  2016
2017     Zwolle     2017    1443.0     977.0    126116.0  2017
2018     Zwolle     2018    1458.0     988.0    127497.0  2018
2019     Zwolle     2019    1460.0     953.0    128840.0  2019

[10944 rows x 6 columns]


In [8]:
print('Calculating percentage deceased of population')
percentage(population,'Deceased','Population')
percentage(population_y,'Deceased', 'Population')
print('DONE')

Calculating percentage deceased of population
DONE


In [9]:
print('Ceating dataframe for Nederland')
population_nl = population[population['RegioS'] == 'Nederland']
population = population[population['RegioS'] != 'Nederland']
population_y_nl = population_y[population_y['RegioS'] == 'Nederland']
population_y = population_y[population_y['RegioS'] != 'Nederland']
print('DONE')

Ceating dataframe for Nederland
DONE


In [1]:
population_nl.plot(subplots=True,figsize=(20,10))
population_nl.pivot(index='month',columns='year', values='Deceased').plot(figsize=(20,10))
population_nl.pivot(index='month',columns='year',values='Deceased_perc').plot(figsize=(20,10))

NameError: name 'population_nl' is not defined

In [ ]:
x=population_y_nl.index
print(x)
ax=population_y_nl.plot(kind='line',x=x, y='Deceased', color='DarkBlue',figsize=(20,10))
ax2=population_y_nl.plot(kind='line',y='Deceased_perc', secondary_y=True,color='Red', ax=ax)
ax.set_ylabel('Deceased')
ax2.set_ylabel('% of population')
plt.tight_layout()
plt.show()

In [ ]:
#print('Ceating dataframe for Landsdeel')
print('SKIPPING Landsdeel')
#population_ld = population[population['RegioS'].str.slice(start=-4) == '(LD)']
population = population[population['RegioS'].str.slice(start=-4) != '(LD)']
#print('DONE')

print('Ceating dataframe for Provincies')
#print('SKIPPING Province')
population_pv = population[population['RegioS'].str.slice(start=-4) == '(PV)']
population_pv['RegioS'] = population_pv['RegioS'].str.slice(stop=-5)
population = population[population['RegioS'].str.slice(start=-4) != '(PV)']
print('DONE')

print('Ceating dataframe for COROP-gebied')
print('SKIPPING COROP-gebied')
#population_cr = population[population['RegioS'].str.slice(start=-4) == '(CR)']
population = population[population['RegioS'].str.slice(start=-4) != '(CR)']
#print('DONE')

print('Ceating dataframe for Gemeenten')
population_gm = population
print('DONE')
#del(population)

In [ ]:
population_pv.pivot(columns='RegioS',values='Deceased_perc').plot(figsize=(20,10))
df = population_gm[population_gm.index == '2020']
df = df.pivot(columns='RegioS',values='Deceased_perc')
df
#df.plot(figsize=(20,40))

In [ ]:
#TO DO
# add CBS oversterfte (look for source)
# causes of death: 7052_95, 82899NED, 7233, 71426ned/71936ned verkeer,84726NED, 37360ned age, 70895ned, 7022gza, 80202ned, 71044NED, 71045NED
# age of death 37979ned, 83194NED, 70895ned
# 83110NED

### and then reproducting the coronadashboard
The data I use is the sources used formhet Coronadashbord of the Dutch government. https://coronadashboard.rijksoverheid.nl/
  
This contains:
*medical indicators*

In [ ]:
print("- number of positive tests")
# number of positive tests (https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427)
positive_tests = pd.read_json('https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.json')
positive_tests.plot()

In [ ]:
print('Date of report for RIVM positive tests data = ' + str(positive_tests.loc[0,'Date_of_report']))
positive_tests = positive_tests.drop(columns='Date_of_report')

In [ ]:
positive_tests = positive_tests.set_index(pd.PeriodIndex(positive_tests['Date_of_publication'], freq='d'))
positive_tests = positive_tests.drop(columns='Date_of_publication')
positive_tests['P'] = positive_tests.index.strftime('%Y-%m')
positive_tests = positive_tests.rename(index={'Date_of_publication':'date'})
positive_tests

In [ ]:
positive_tests_nl = positive_tests.sum(level='Date_of_publication')
positive_tests_nl['P'] = positive_tests_nl.index.strftime('%Y-%m')
positive_tests_nl

In [ ]:
pop_latest = population_nl.loc[population_nl['P'].max(),'Population']
pop_latest

In [ ]:
print(type(population_nl.P[0]))
print(type(positive_tests_nl.P[0]))

pd.merge_asof(positive_tests_nl,population_nl,left_on='P',right_on='P')

# Gemiddeld aantal positief geteste mensen per 100.000 inwoners
## Dit getal laat zien van hoeveel mensen gisteren per 100.000 inwoners gemeld is dat ze positief getest zijn en COVID-19 hebben.
positive_tests_nl['p100k'] = positive_tests_nl['Total_reported'] / population_nl.loc[if(positive_tests_nl['P'] > population_nl['P'].max()):
                                                                                     population_nl.loc[positive_tests_nl['P'],'Population']
                                                                                else:
                                                                                     positive_tests_nl['P']
                                                                                ,'Population'] * 100000


#pt_agg['mov_avg_p100k'] = np.average(pt_agg['']])
#np.mean(pt[len(pt.columns)-1].Total_reported)

In [ ]:
# Aantal positief geteste mensen
## Dit getal laat zien van hoeveel mensen gisteren gemeld is dat ze positief getest zijn en COVID-19 hebben.

In [ ]:
#Verdeling positief geteste mensen in Nederland
## per gemeente

## per veiligheidsregio



# TESTING BOKEH WITH OTHER DATA
pt_agg = pt.groupby(by=['Date_of_report'],as_index=False).sum()

q = figure(plot_width=800, plot_height=400, title="My Line Plot", x_axis_type="datetime")

q.multi_line(
    xs=[
        pt_agg['Date_of_report'],
        pt_agg['Date_of_report'],
        pt_agg['Date_of_report']
    ],
    ys=[
        pt_agg['Deceased'], 
        pt_agg['Hospital_admission'],
        pt_agg['Total_reported']
    ],
    color=viridis(len(pt_agg.columns)-1)
)

show(q)

In [ ]:
print("- Percentage positive tested of all tests done")
# https://www.rivm.nl/archief-weekrapportages-covid-19-in-nederland
pt_perc = tabula.read_pdf('https://www.rivm.nl/sites/default/files/2020-10/COVID-19_WebSite_rapport_wekelijks_20201013_1159_0.pdf', pages=29, guess=True, stream=True)
print(pt_perc)

## To DO:
## 1) extract data from latest pdf
## 2) scrape all pdfs for data to compare data thought time

In [ ]:
print("- number of infectious people")
# https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/097155aa-75eb-4caa-8ed3-4c6edb80467e
infected = pd.read_json('https://data.rivm.nl/covid-19/COVID-19_prevalentie.json')
print(infected.head())

In [ ]:
print("- R (reproductionnumber)")
# https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/ed0699d1-c9d5-4436-8517-27eb993eab6e
R = pd.read_json('https://data.rivm.nl/covid-19/COVID-19_reproductiegetal.json')
print(R.head())

In [ ]:
print("- hospital admissions per day")
# NICE https://www.databronnencovid19.nl/Bron?naam=Nationale-Intensive-Care-Evaluatie
hosp_nice = pd.read_json('https://stichting-nice.nl/covid-19/public/intake-count/')
print(hosp_nice.head())
# LCPS https://lcps.nu/datafeed/
hosp_lcps = pd.read_csv('https://lcps.nu/wp-content/uploads/covid-19.csv')
print(hosp_lcps.head())
# Dashbaord changed source data https://www.nu.nl/coronavirus/6083846/ministerie-meldde-maandenlang-veel-te-weinig-opnames-coronapatienten.html
# other sources: https://www.stichting-nice.nl/covid-19-op-de-zkh.jsp
# 37852, 84522NED

In [ ]:
print("- icu admissions per day")
# url
ic_NICE_new_intake = pd.read_json('https://stichting-nice.nl/covid-19/public/new-intake/').T
ic_NICE_new_intake0 = pd.DataFrame()
for row in ic_NICE_new_intake[0]:
    ic_NICE_new_intake0 = ic_NICE_new_intake0.append(row, ignore_index=True)
print(ic_NICE_new_intake0)
ic_NICE_new_intake0['date'] = ic_NICE_new_intake['date']
ic_NICE_new_intake1 = pd.DataFrame()
for row in ic_NICE_new_intake[1]:
    ic_NICE_new_intake1 = ic_NICE_new_intake1.append(row, ignore_index=True)
ic_NICE_new_intake01 = pd.concat([ic_NICE_new_intake0,ic_NICE_new_intake1])
print(ic_NICE_new_intake01.head())
ic_NICE_new_intake_confirmed = pd.read_json('https://stichting-nice.nl/covid-19/public/new-intake/confirmed/')
#print(ic_NICE_new_intake_confirmed.head())
ic_NICE_intake_count = pd.read_json('https://stichting-nice.nl/covid-19/public/intake-count/')
#print(ic_NICE_intake_count.head())
print('IC data: 3 sources IMPORTED')
## To DO: conpare ic1 and ic2
#other sources: 

In [ ]:
ic_NICE_new_intake0

In [ ]:
# 84069NED

# SINGLE LINE TEST
# create a new plot (with a title) using figure
ic_new_intake0 = figure(plot_width=800, plot_height=400, x_axis_type="datetime", title="NICE new-intake")
# add a line renderer 
ic_new_intake0.line(
    ic_NICE_new_intake0['date'], 
    ic_NICE_new_intake0['value']
)
show(ic_new_intake0)

# SINGLE LINE TEST
# create a new plot (with a title) using figure
ic_new_intake_confirmed = figure(plot_width=800, plot_height=400, x_axis_type="datetime", title="NICE new-intake confirmed")
# add a line renderer 
ic_new_intake_confirmed.line(
    ic_NICE_new_intake_confirmed['date'], 
    ic_NICE_new_intake_confirmed['value']
)

# SINGLE LINE TEST
# create a new plot (with a title) using figure
ic_intake_count = figure(plot_width=800, plot_height=400, x_axis_type="datetime", title="NICE intake count")
# add a line renderer 
ic_intake_count.line(
    ic_NICE_intake_count['date'], 
    ic_NICE_intake_count['value']
)

## MULTIPLE LINE TEST
ic_all = figure(title="IC1 & IC_opnames_NICE", plot_width=800, plot_height=400, x_axis_type="datetime")

ic_all.multi_line([ic1.date, ic_opnames.date],
              [ic1.value, ic_opnames.value],
              color=viridis(3))

show(ic_new_intake)
show(ic_new_intake_confirmed)
show(ic_intake_count)
show(ic_all)

In [ ]:
## sources NICE for their website:
# /covid-19/public/zkh/global
# /covid-19/public/zkh/new-intake/
# /covid-19/public/zkh/intake-count/
# /covid-19/public/zkh/intake-cumulative/
# /covid-19/public/zkh/died-and-survivors-cumulative/
# /covid-19/public/zkh/age-distribution-died-and-survivors/
# /covid-19/public/zkh/age-distribution-status/
# /covid-19/public/zkh/behandelduur-distribution/
# /covid-19/public/zkh/behandelduur-distribution/
## TO DO: check which used and which to use extra.
## sources LCPS?
# https://lcps.nu/wp-content/uploads/
## Maybe scrape website to find more public data: https://scrapy.org/ OR https://www.crummy.com/software/BeautifulSoup/bs4/doc/

*early signs*

In [ ]:
print("- patient reporting covid symptoms at family doctor")
# https://www.nivel.nl/nl/nivel-zorgregistraties-eerste-lijn/nivel-zorgregistraties-eerste-lijn


In [ ]:
print("- sewagewate")
# https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/a2960b68-9d3f-4dc3-9485-600570cd52b9
sw = pd.read_json('https://data.rivm.nl/covid-19/COVID-19_rioolwaterdata.json')
print(sw.head())

*homes for elderly*

In [ ]:
print("- positive testst")
# from pdf. https://www.rivm.nl/documenten/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland

In [ ]:
print("- infected locations")
# from pdf. https://www.rivm.nl/documenten/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland

In [ ]:
print("- number deseased")
# from pdf. https://www.rivm.nl/documenten/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland

**metadata**

In [ ]:
print("- municipalities in Netherlands")
## https://www.cbs.nl/nl-nl/onze-diensten/methoden/classificaties/overig/gemeentelijke-indelingen-per-jaar/indeling%20per%20jaar/gemeentelijke-indeling-op-1-januari-2020
municipality = pd.read_excel('https://www.cbs.nl/-/media/_excel/2020/03/gemeenten-alfabetisch-2020.xlsx')
print(municipality.head())
# aantallen per gemeente: https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427


### missing data, i would like to calculate of collect
- covid numbers relitive to all data
- deseased for whole country (possible source: https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/1c0fcd57-1102-4620-9cfa-441e93ea5604)

### more data

- Karakteristieken elke geteste persoon: https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/2c4357c8-76e4-4662-9574-1deb8a73f724?tab=relations
- https://data.rivm.nl

In [ ]:
#check data collection
from pivottablejs import pivot_ui
from IPython.display import HTML

df_to_check = ic1

pivot_ui(df_to_check, outfile_path='pivottablejs.html')
HTML('pivottablejs.html')